In [5]:
import torch
import torch.nn as nn
import torch.optim as optim
from torch.utils.data import DataLoader
from torchvision import datasets, transforms, models
import os


# ===========================
# 1. 配置 GPU/CPU
# ===========================
device = torch.device("cuda" if torch.cuda.is_available() else "cpu")
print("Using device:", device)

# ===========================
# 2. 数据增强
# ===========================
train_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.RandomHorizontalFlip(),
    transforms.RandomRotation(15),
    transforms.RandomResizedCrop(224, scale=(0.8, 1.0)),
    transforms.ColorJitter(brightness=0.3, contrast=0.3, saturation=0.3),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

val_transform = transforms.Compose([
    transforms.Resize((224, 224)),
    transforms.CenterCrop(224),
    transforms.ToTensor(),
    transforms.Normalize([0.485, 0.456, 0.406],
                         [0.229, 0.224, 0.225])
])

# ===========================
# 3. 加载数据集
# ===========================
train_dataset = datasets.ImageFolder("./cats_and_dogs/train", transform=train_transform)
val_dataset   = datasets.ImageFolder("./cats_and_dogs/val",   transform=val_transform)

train_loader = DataLoader(train_dataset, batch_size=32, shuffle=True, num_workers=0)
val_loader   = DataLoader(val_dataset,   batch_size=32, shuffle=False, num_workers=0)

# ===========================
# 4. 加载 ResNet18 预训练模型
# ===========================
model = models.resnet18(weights=models.ResNet18_Weights.IMAGENET1K_V1)
model.fc = nn.Linear(512, 2)  # 2 类：cats / dogs
model = model.to(device)

# ===========================
# 5. 损失函数与优化器
# ===========================
criterion = nn.CrossEntropyLoss()
optimizer = optim.Adam(model.parameters(), lr=0.0003)
scheduler = optim.lr_scheduler.StepLR(optimizer, step_size=10, gamma=0.5)

# ===========================
# 6. Early Stopping 配置
# ===========================
best_val_acc = 0
patience, wait = 10, 0

# ===========================
# 7. 训练函数
# ===========================
def evaluate(model):
    model.eval()
    correct, total = 0, 0
    with torch.no_grad():
        for imgs, labels in val_loader:
            imgs, labels = imgs.to(device), labels.to(device)
            outputs = model(imgs)
            correct += (outputs.argmax(1) == labels).sum().item()
            total += labels.size(0)
    return correct / total


# ===========================
# 8. 主训练循环
# ===========================
for epoch in range(1, 101):
    model.train()
    correct, total = 0, 0

    for imgs, labels in train_loader:
        imgs, labels = imgs.to(device), labels.to(device)

        optimizer.zero_grad()
        outputs = model(imgs)
        loss = criterion(outputs, labels)
        loss.backward()
        optimizer.step()

        correct += (outputs.argmax(1) == labels).sum().item()
        total += labels.size(0)

    train_acc = correct / total
    val_acc = evaluate(model)
    scheduler.step()

    print(f"Epoch {epoch}/100, Train Acc: {train_acc:.4f}, Val Acc: {val_acc:.4f}")

    # 保存最佳模型
    if val_acc > best_val_acc:
        best_val_acc = val_acc
        torch.save(model.state_dict(), "best.pth")
        wait = 0
        print("🔥 Saved best model!")
    else:
        wait += 1

    # Early stopping
    if wait >= patience:
        print("⛔ Early stopping triggered!")
        break

print(f"Training finished! Best Val Acc = {best_val_acc:.4f}")


Using device: cpu
Downloading: "https://download.pytorch.org/models/resnet18-f37072fd.pth" to C:\Users\32383/.cache\torch\hub\checkpoints\resnet18-f37072fd.pth


100.0%


Epoch 1/100, Train Acc: 0.9010, Val Acc: 0.8200
🔥 Saved best model!
Epoch 2/100, Train Acc: 0.9550, Val Acc: 0.9700
🔥 Saved best model!
Epoch 3/100, Train Acc: 0.9630, Val Acc: 0.9675
Epoch 4/100, Train Acc: 0.9790, Val Acc: 0.9625
Epoch 5/100, Train Acc: 0.9820, Val Acc: 0.9625
Epoch 6/100, Train Acc: 0.9840, Val Acc: 0.9850
🔥 Saved best model!
Epoch 7/100, Train Acc: 0.9850, Val Acc: 0.9800
Epoch 8/100, Train Acc: 0.9850, Val Acc: 0.9700
Epoch 9/100, Train Acc: 0.9720, Val Acc: 0.9550
Epoch 10/100, Train Acc: 0.9810, Val Acc: 0.9675
Epoch 11/100, Train Acc: 0.9960, Val Acc: 0.9750
Epoch 12/100, Train Acc: 0.9910, Val Acc: 0.9700
Epoch 13/100, Train Acc: 0.9920, Val Acc: 0.9725
Epoch 14/100, Train Acc: 0.9940, Val Acc: 0.9725
Epoch 15/100, Train Acc: 0.9850, Val Acc: 0.9625
Epoch 16/100, Train Acc: 0.9980, Val Acc: 0.9800
⛔ Early stopping triggered!
Training finished! Best Val Acc = 0.9850
